In [2]:
import numpy as np
import cv2
import math
import collections
import os
from pathlib import Path


In [3]:
def red_filter(img):
    lower = np.array([0, 0, 90])
    upper = np.array([65, 50, 255])
    mask = cv2.inRange(img, lower, upper)
    return mask

def yellow_filter(img):
    img = cv2.split(img)[1]
    img = cv2.medianBlur(img,5)
    mask = cv2.threshold(img, 110, 255, cv2.THRESH_BINARY)[1]
    return mask

def blue_filter(img):
    lower = np.array([39, 40, 41])
    upper = np.array([125, 140, 150])
    mask = cv2.inRange(img, lower, upper)
    return mask

def blue_filter2(img):
    img = cv2.split(img)[0]
    img = cv2.medianBlur(img,5)
    mask = np.array(cv2.threshold(img, 50, 255, cv2.THRESH_BINARY)[1])
    return mask

In [4]:
def hexagon_contours(img):
    img = cv2.medianBlur(img, 5)
    bin_img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 19, 2)
    bin_img = cv2.medianBlur(bin_img, 5)
    contours=cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    external = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    max_ext_area =np.max([cv2.contourArea(x) for x in external])
    areas=[]
    for x in contours:
        area=cv2.contourArea(x)
        if area<max_ext_area:
            areas.append(area)
    max_inner_area =np.max(areas)
    hex_list=[]
    for x in contours:
        area=cv2.contourArea(x)
        if 0.5*max_inner_area<area<max_ext_area:
            hexagon = cv2.convexHull(x)
            epsilon = 0.05*(0.5*max_inner_area)/area * cv2.arcLength(hexagon, True)
            hexagon = cv2.approxPolyDP(hexagon, epsilon, True)
            if len(hexagon)==6:
                hex_list.append(hexagon)
    return hex_list

In [5]:
import matplotlib.pyplot as plt

def plot_image(img, zoom=1.5):
    assert(2 <= len(img.shape) <= 3)
    
    is_gray = len(img.shape) == 2
    if not is_gray:
        img = img[:,:,::-1]
    n_len = 6.5
    
    n, m = img.shape[:2]
    frac = n / float(m)
    n, m = zoom * frac * n_len, zoom * n_len
    
    fig = plt.figure(figsize=(n, m))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray' if is_gray else None)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.show()

In [6]:
def read(pattern="Group_?.bmp"):
    images_0 = []
    images_1 = []
    
    folder = Path("samples")
    files_with_maps = folder.glob(pattern)
    files=list(files_with_maps)
    
    for file in files:
        img = cv2.imread(str(file), 0)
        images_0.append(img)
        
        img = cv2.imread(str(file), 1)
        images_1.append(img)
        
    return images_0, images_1



In [7]:
def solution(folder, train, images_0, images_1, clfs = None):
    if train:
        fracs_red =[]
        fracs_blue = []
        fracs_blue2 = []
        fracs_yellow = []    
    for k in range(len(images_0)):
        hex_list=hexagon_contours(images_0[k])
        for hexagon in hex_list:
            M = cv2.moments(hexagon)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            if train is False:
                descr = ["" for x in range(len(clfs))]
            for lin in range(6):
                red  = red_filter(images_1[k]).copy()
                blue = blue_filter(images_1[k]).copy()
                blue2 = blue_filter2(images_1[k]).copy()
                yellow = yellow_filter(images_1[k]).copy()
                x1 = hexagon[lin % 6][0][0]
                x2 = hexagon[(lin+1)%6][0][0]
                y2 = hexagon[(lin+1)%6][0][1]
                y1 = hexagon[lin%6][0][1]
                rads = math.atan2(y2 - y1, x2 - x1)
                if rads < 0:
                    rads = rads + 2 * math.pi
                degs = math.degrees(rads)                
                center = (int((x1+x2)/2),int((y1+y2)/2))
                length = int(math.sqrt((x1-x2)**2 + (y1-y2)**2))
                mask = np.zeros(images_0[k].shape)
                cv2.ellipse(mask, center, (length//3, length//3), degs, 0, 180, 1, -1)
                ellipse_area = np.sum(mask!=0)
                red[mask==0] = 0
                blue[mask == 0] = 0
                blue2[mask == 0] = 0
                yellow[mask == 0] = 0
                area_red = np.sum(red!=0)
                area_blue2 = np.sum(blue2!=0)
                area_blue = np.sum(blue != 0)
                area_yellow = np.sum(yellow!=0)
                frac_red = round(float(area_red)/ellipse_area,2)
                frac_blue = round(float(area_blue)/ellipse_area,2)
                frac_blue2 = round(float(area_blue2) / ellipse_area, 2)
                frac_yellow = round(float(area_yellow)/ellipse_area,2)
                if train:
                    fracs_red.append(frac_red)
                    fracs_blue.append(frac_blue)
                    fracs_blue2.append(frac_blue2)
                    fracs_yellow.append(frac_yellow)
                else:
                    for i in range(len(clfs)):
                        descr[i]+=str(clfs[i].predict(np.array([frac_yellow,frac_blue,frac_blue2,frac_red]).reshape(1, -1))[0])
            if train is False:
                for i in range(len(clfs)):
                    s = descr[i]
                    text = ""
                    for n in range(10):
                        if s in col_cycles_all[n]:
                            text = str(numbers[n])
                            break;
                    if text != "":
                        cv2.putText(images_1[k], text, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, 0)
                        break;
            
        if train is False:
            cv2.imwrite(folder+"\images_"  + str(k) + ".jpg", images_1[k])
        else:
            return fracs_yellow, fracs_blue, fracs_blue2, fracs_red

In [8]:
#Обучение:
image_0, image_1=read("Dozen_0.bmp")
fracs_yellow, fracs_blue, fracs_blue2, fracs_red=solution("dozen_results",True,image_0,image_1)

In [14]:
len(fracs_yellow)

60

In [9]:
ans = list("yybrbrrrbyybyyrbrbbyrybrrbbyyrbbryyrbbryrybryrbyrybyrbbbyryr")
col_cycles_all = []
numbers = [1,2,10,9,3,5,8,6,4,7]
for k in range(10):
    col_cycle =[]
    seq = np.array((ans[6*k:6*(k+1)]))
    col_cycle.append("".join(seq))
    deq = collections.deque(seq)
    for k in range(5):
        deq.rotate(1)
        col_cycle.append("".join(deq))
    col_cycles_all.append(col_cycle)

import pandas as pd
data = pd.DataFrame(ans)
data["yellow"] = fracs_yellow
data["blue"] = fracs_blue
data["blue2"] = fracs_blue2
data["red"] = fracs_red
y = data[0]
X = data[["yellow","blue","blue2","red"]]



In [18]:
y

0     y
1     y
2     b
3     r
4     b
5     r
6     r
7     r
8     b
9     y
10    y
11    b
12    y
13    y
14    r
15    b
16    r
17    b
18    b
19    y
20    r
21    y
22    b
23    r
24    r
25    b
26    b
27    y
28    y
29    r
30    b
31    b
32    r
33    y
34    y
35    r
36    b
37    b
38    r
39    y
40    r
41    y
42    b
43    r
44    y
45    r
46    b
47    y
48    r
49    y
50    b
51    y
52    r
53    b
54    b
55    b
56    y
57    r
58    y
59    r
Name: 0, dtype: object

In [15]:
#Классификаторы
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()
clf1.fit(X,y)
from sklearn.ensemble import GradientBoostingClassifier
clf2 = GradientBoostingClassifier()
clf2.fit(X,y)
from sklearn.neighbors import KNeighborsClassifier
clf3 = KNeighborsClassifier()
clf3.fit(X,y)
from sklearn import svm
clf4 = svm.SVC()
clf4.fit(X, y)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [16]:
#Задача 3
image_0, image_1=read("Single_?.bmp")
solution("single_results",False,image_0,image_1,[clf1,clf2,clf3,clf4])

In [47]:
#Задача 4
image_0, image_1=read("Group_?.bmp")
solution("group_results",False,image_0,image_1,[clf1,clf2,clf3,clf4])